In [1]:
import os
import json
from groq import Groq
from openai import OpenAI

In [2]:
#open dataset and store it as a JSON object
with open('../GSM-IC/GSM-IC_mstep.json', 'r') as file:
    questions = json.load(file)

In [3]:
#example of an entry in the dataset
#todo: what's the point of all the keys under 'n_steps'?
questions[0]

{'original_question': 'Officer Hopps has to give out 200 tickets in May. The first 15 days he averages 8 tickets a day. How many does he have to average each day for the rest of the month to reach his required goal?',
 'answer': '5',
 'new_question': "Officer Hopps has to give out 200 tickets in May. The first 15 days he averages 8 tickets a day. Officer Hopps' mother bought 200 bus tickets in Feburary. How many does he have to average each day for the rest of the month to reach his required goal?",
 'n_steps': 4,
 'role': "Officer Hopps' mother",
 'number': '200',
 'sentence_template': '{role} bought {number} bus tickets in Feburary.',
 'role_label': 'overlapped',
 'number_label': 'in_range',
 'sentence_label': 'in_topic'}

In [4]:
#initialize the Groq client. This allows us to query Llama3 (or another model) from my Groq account
#todo: store the api_key in a secure way!
# client = Groq(
#     # api_key=os.environ.get("GROQ_API_KEY")
#     api_key='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'
# )
client = OpenAI(api_key=open("api.txt").read())

In [5]:
#todo: 1. answer_prompt is the current way of forcing the model to output its answer in predictable format. Find a better way to extract the numerical answer from the model's output. (maybe: require JSON output)

#todo: 2. clean up the answer. Ex. remove $, %, inches, etc. 

def single_query(query):
    """
    query the model with a single question
    input: what you would type in the chat box to ask the model a question
    output: the model's output in response to your query 
    """

    
    answer_prompt='The last sentence of your response must be in the form "The answer is [your answer]".'
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query + answer_prompt,
        }
    ],
    model="gpt-3.5-turbo-0125",
    )
    response = chat_completion.choices[0].message.content
    return response 


In [6]:
#todo: shouldn't need to separate the two queries like this
def multiple_queries(question_type='original_question', num_questions=5):
    """
    query the model sequentially with multiple questions from the dataset 
    Args: question_type: specify whether to query the model with the original question or the new question 
        num_questions: the number of questions (mainly for testing a few questions at a time)
    
    """
    for idx, question in enumerate(questions):
        if idx < num_questions:
            query = question[question_type]
            response = single_query(query)
            print(response)
            print('--------------------------------------------------')

In [7]:
#query with relevant information only
multiple_queries(question_type='original_question', num_questions=5)


Officer Hopps has given out 120 tickets in the first 15 days of May. Therefore, he needs to give out 80 more tickets in the remaining 16 days of the month. To reach his goal, he needs to average 5 tickets per day for the rest of the month.

The answer is 5 tickets per day.
--------------------------------------------------
Each box costs $15/3 = $5.
Grover has a total of 3 boxes x 20 face masks = 60 face masks.
If he sells each face mask for $0.50, then his total revenue will be 60 x $0.50 = $30.
His total profit will be $30 - $15 = $15.

The answer is $15.
--------------------------------------------------
In the first round, Sam completed 16 skips, and Jeff completed 15 skips.
In the second round, Sam completed 16 skips, and Jeff completed 13 skips.
In the third round, Sam completed 16 skips, and Jeff completed 20 skips.
In the last round, Sam completed 16 skips, and Jeff completed 8 skips.

So, Jeff completed an average of (15 + 13 + 20 + 8) / 4 = 14 skips per round. 

The answer is

In [8]:
#query with irrelevant information included
#todo: merge with querying 'original_question' to more easily compare differences in model output
multiple_queries(question_type='new_question', num_questions=5)

Officer Hopps gave out 120 tickets in the first 15 days (15 days * 8 tickets/day = 120 tickets). He needs to give out 80 more tickets to reach his goal of 200 tickets in May (200 tickets - 120 tickets = 80 tickets). There are 16 days remaining in May (31 days total - 15 days = 16 days). Therefore, Officer Hopps needs to average 5 tickets per day for the rest of the month (80 tickets / 16 days = 5 tickets/day).

The answer is 5.
--------------------------------------------------
First, let's calculate the total cost of the face masks Grover bought. Since each box has 20 face masks and he bought 3 boxes, then Grover has 3 x 20 = 60 face masks in total.

If Grover bought the 3 boxes for $15, then the cost per face mask is $15 / 60 masks = $0.25 per mask.

Grover plans to sell each mask for $0.50, so his profit per mask would be $0.50 - $0.25 = $0.25 per mask.

If Grover sells all 60 masks, his total profit would be 60 masks x $0.25 = $15.

Therefore, Grover's total profit will be $15.

Th

In [9]:
#todo: shouldn't need to separate the two queries like this
def get_answers(question_type='original_question', num_questions=5):
    """
    query the model sequentially with multiple questions from the dataset 
    Args: question_type: specify whether to query the model with the original question or the new question 
        num_questions: the number of questions (mainly for testing a few questions at a time)
    
    """
    for idx, question in enumerate(questions):
        if idx < num_questions:
            query = question[question_type]
            response = single_query(query)
            # print(response)
            try:
                answer = response.split("he answer is")[1].strip("$")
                print(question['answer'], answer)
            except:
                print(response, question['answer'])
                # input()

In [10]:
get_answers(question_type='new_question', num_questions=100)

5  5.
15  -$15.
14  14 skips.
325  285 beads.
20  20 pages.
105  105.
25  5 baskets.
9  9.
180  180 kilometers.
3  4.
134  134 pounds.
12  12.
64  64 ounces.
59  $54.
3  6.
136  146.
5  5.
15  15.
57  $57.
25  25 baskets.
15  $15.
9  9.
195  195.
90  90.
15  $15.
80  80kg.
40  32.
62  62.
4  4.
48  13.
39  39.
48  19.
40  40.
4100  $4100.
62  62 pounds.
59  $59.
9  9.
4  4 hours.
400  270 ml.
20  45.
1200  1200.
3  4.
325  325 beads.
59  $59.
325  285 beads.
300  300.
4  4 hours.
325  285 beads.
90  90.
3  5.
62  62.
96  $224.
195  195.
4100  $6100.
96  $64.
21  17.
188  211.
62  62 pounds.
9  9.
40  40.
12  12.
40  40.
15  0.
14  14.
4  4 hours.
32  32.
57  $57.
15  38.
4100  $4100.
5  5.33.
188  62.
12  12.
32  32 beads.
14  14.
134  134 pounds.
3  4.
9  9.
25  25.
325  285 beads.
32  32 beads.
1036  $1036.00.
4  5.
39  39 balloons.
40  40.
80  80kg.
48  15.
195  195.
57  $57.
15  $15.
62  62.
62  62 pounds.
195  195.
9  9.
1200  1200.
3  23.
45  65.
57  $57.
105  99.
105  108.
105  